In [20]:
import yfinance as yf
import pandas as pd 
from sec_cik_mapper import StockMapper
from dataclasses import dataclass
from datetime import datetime, timedelta
import logging 
from scipy.optimize import minimize
import numpy as np

# Setup logging
logging.basicConfig(level=logging.INFO)

# Define the month codes used in futures tickers (e.g., F for January, G for February)
months = {
    "F": "Jan", "G": "Feb", "H": "Mar", "J": "Apr",
    "K": "May", "M": "Jun", "N": "Jul", "Q": "Aug",
    "U": "Sep", "V": "Oct", "X": "Nov", "Z": "Dec"
}

#---------------------------------------------------------
# Functions
#---------------------------------------------------------

# Function to retrieve historical data for a single commodity
def get_commodity_data(ticker, start_date, end_date):
    """
    Retrieves historical data on prices for a given commodity.

    Args:
        ticker (str): The commodity ticker
        start_date (str): Start date in the format 'YYYY-MM-DD'
        end_date (str): End date in the format 'YYYY-MM-DD'

    Returns:
        pd.DataFrame: A pandas dataframe with the historical data

    Example:
        df = get_commodity_data('CL=F', '2020-01-01', '2020-12-31')
    """
    try:
        commodity = yf.Ticker(ticker)
        data = commodity.history(start=start_date, end=end_date, auto_adjust=False, actions=False)
        df = pd.DataFrame(data)
        df['ticker'] = ticker
        df.reset_index(inplace=True)
        return df
    except Exception as e:
        logging.warning(f"Error retrieving data for {ticker}: {e}")
        return pd.DataFrame()

# Function to retrieve historical data for multiple commodities
def get_commodities_data(tickers, start_date, end_date):
    """
    Retrieves historical data on prices for a list of commodities.

    Args:
        tickers (list): List of commodity tickers
        start_date (str): Start date in the format 'YYYY-MM-DD'
        end_date (str): End date in the format 'YYYY-MM-DD'

    Returns:
        pd.DataFrame: A pandas dataframe with the historical data

    Example:
        df = get_commodities_data(['CL=F', 'NG=F'], '2020-01-01', '2020-12-31')
    """
    dfs = []
    for ticker in tickers:
        try:
            df = get_commodity_data(ticker, start_date, end_date)
            if not df.empty:
                dfs.append(df)
        except Exception as e:
            logging.warning(f"Commodity {ticker} not found: {e}")
    # Concatenate all dataframes
    data = pd.concat(dfs, ignore_index=True)
    return data

#---------------------------------------------------------
# Example Usage
#---------------------------------------------------------

# Example commodity tickers
commodity_tickers = ['CL=F']  # Crude Oil and Natural Gas

# Get historical data for commodities
start_date = '2000-01-01'
end_date = '2023-12-31'

commodities_data = get_commodities_data(commodity_tickers, start_date, end_date)
print(commodities_data)

#---------------------------------------------------------
# Commodities tickers
#---------------------------------------------------------

# Soft Commodities


                          Date       Open       High        Low      Close  \
0    2000-08-23 00:00:00-04:00  31.950001  32.799999  31.950001  32.049999   
1    2000-08-24 00:00:00-04:00  31.900000  32.240002  31.400000  31.629999   
2    2000-08-25 00:00:00-04:00  31.700001  32.099998  31.320000  32.049999   
3    2000-08-28 00:00:00-04:00  32.040001  32.919998  31.860001  32.869999   
4    2000-08-29 00:00:00-04:00  32.820000  33.029999  32.560001  32.720001   
...                        ...        ...        ...        ...        ...   
5858 2023-12-22 00:00:00-05:00  73.910004  74.980003  73.389999  73.559998   
5859 2023-12-26 00:00:00-05:00  73.559998  76.180000  73.129997  75.570000   
5860 2023-12-27 00:00:00-05:00  75.320000  75.660004  73.769997  74.110001   
5861 2023-12-28 00:00:00-05:00  73.800003  74.400002  71.720001  71.769997   
5862 2023-12-29 00:00:00-05:00  71.989998  72.620003  71.250000  71.650002   

      Adj Close  Volume ticker  
0     32.049999   79385   CL=F

In [23]:
import datetime

# Timestamp from the data
expire_date_timestamp = 1737417600

# Convert to a readable date
expire_date = datetime.datetime.utcfromtimestamp(expire_date_timestamp)
print(f"Expiration Date: {expire_date}")

Expiration Date: 2025-01-21 00:00:00


In [86]:
import yfinance as yf
from datetime import datetime

def get_expiry_date_from_yfinance(ticker):
    """
    Fetch the expiry date of a futures contract from Yahoo Finance.

    Args:
        ticker (str): The ticker symbol of the contract (e.g., "CL=F").

    Returns:
        datetime or None: Expiry date if available, None otherwise.
    """
    try:
        # Initialize the Ticker object
        contract = yf.Ticker(ticker)
        
        # Retrieve contract information
        info = contract.info
        
        # Extract the expiry date if available
        if 'expireDate' in info:
            expiry_timestamp = info['expireDate']
            expiry_date = datetime.utcfromtimestamp(expiry_timestamp)  # Corrected usage
            return expiry_date
        else:
            print(f"No expiry date found for {ticker}.")
            return None
    except Exception as e:
        print(f"Error retrieving expiry date for {ticker}: {e}")
        return None

# Example usage
ticker = "NG=F"
expiry_date = get_expiry_date_from_yfinance(ticker)
if expiry_date:
    print(f"The expiry date for {ticker} is {expiry_date.date()}.")
else:
    print(f"No expiry date available for {ticker}.")

# ZS=F --> Soybean, ZC=F --> Corn, ZW=F --> Wheat, CC=F --> Cocoa

# Energy Commodities
# CL=F --> Crude (WTI), BZ=F --> Crude (Brent), NG=F --> Natural Gas, HO=F --> Heating Oil

# Metals 
# GC=F --> Gold, SI=F --> Silver, HG = F --> Copper

The expiry date for NG=F is 2025-01-29.


In [60]:
from datetime import datetime, timedelta

def get_cbot_expiry(buy_date, commodity):
    """
    Calculate the expiry date of the front-month futures contract for CBOT commodities based on the date of purchase.

    Args:
        buy_date (str): The purchase date in 'YYYY-MM-DD' format.
        commodity (str): The commodity type ('soybean', 'wheat', 'corn', 'cocoa').

    Returns:
        str: Expiry date of the front-month contract in 'YYYY-MM-DD' format.

    Logic:
        1. Identify valid contract months for the specified commodity.
        2. Determine the front-month contract based on the purchase date.
        3. Calculate the expiry date for the front-month contract using the rule:
           "Business day before the 15th of the contract month."
        4. Define a rollover period (20 business days before expiry) to simulate realistic market behavior.
        5. If the purchase date falls within the rollover period, roll to the next contract.
    """
    # Define valid contract months for each commodity
    cbot_contract_months = {
        'soybean': [1, 3, 5, 7, 8, 9, 11],  # January, March, May, July, August, September, November
        'wheat': [3, 5, 7, 9, 12],          # March, May, July, September, December
        'corn': [3, 5, 7, 9, 12],           # March, May, July, September, December
        'cocoa': [3, 5, 7, 9, 12],          # March, May, July, September, December
    }

    try:
        # Convert input date to datetime object
        buy_date = datetime.strptime(buy_date, '%Y-%m-%d')

        # Get the contract months for the selected commodity
        if commodity not in cbot_contract_months:
            raise ValueError(f"Unsupported commodity: {commodity}. Supported commodities are: {', '.join(cbot_contract_months.keys())}")

        contract_months = cbot_contract_months[commodity]

        # Determine the front-month contract
        year = buy_date.year
        month = buy_date.month

        # Find the nearest valid contract month
        for contract_month in contract_months:
            if contract_month >= month:
                front_month = contract_month
                break
        else:
            # If no valid month in current year, roll to the next year's first contract month
            front_month = contract_months[0]
            year += 1

        # Calculate the expiry date for the front-month contract
        expiry_date = datetime(year, front_month, 15) - timedelta(days=1)
        while expiry_date.weekday() in (5, 6):  # Ensure expiry date is a business day
            expiry_date -= timedelta(days=1)

        # Define a rollover period (e.g., 20 business days before expiry)
        rollover_date = expiry_date
        business_days = 0
        while business_days < 20:
            rollover_date -= timedelta(days=1)
            if rollover_date.weekday() < 5:  # Count only weekdays (Monday-Friday)
                business_days += 1

        # If buy_date is on or after the rollover date, move to the next contract
        if buy_date >= rollover_date:
            next_index = contract_months.index(front_month) + 1
            if next_index >= len(contract_months):
                front_month = contract_months[0]
                year += 1
            else:
                front_month = contract_months[next_index]

            # Recalculate expiry date for the new contract
            expiry_date = datetime(year, front_month, 15) - timedelta(days=1)
            while expiry_date.weekday() in (5, 6):
                expiry_date -= timedelta(days=1)

        return expiry_date.strftime('%Y-%m-%d')

    except Exception as e:
        return f"Error calculating expiry date: {e}"

# Example usage
buy_date = "2024-12-29"  # Replace with the date of purchase
commodity = "soybean"  # Replace with 'wheat', 'corn', or 'cocoa'
expiry_date = get_cbot_expiry(buy_date, commodity)
print(f"The expiry date for the front-month {commodity} futures contract purchased on {buy_date} is {expiry_date}.")


The expiry date for the front-month soybean futures contract purchased on 2024-12-29 is 2025-03-14.


In [87]:
from datetime import datetime
import pandas as pd

def get_energy_expiry(buy_date, ticker):
    """
    Calculate the expiry date of the front-month futures contract for energy commodities based on the date of purchase.

    Args:
        buy_date (str): The purchase date in 'YYYY-MM-DD' format.
        ticker (str): The commodity ticker ('CL=F', 'BZ=F', 'NG=F', 'HO=F').

    Returns:
        str: Expiry date of the front-month contract in 'YYYY-MM-DD' format.

    Logic:
        1. Identify the commodity based on the ticker.
        2. Define expiry rules for the identified commodity.
        3. Determine delivery month based on purchase date.
        4. Handle cases where the expiry of the next delivery month is before the buy date.
        5. Calculate expiry date using specific commodity rules.
        6. Adjust expiry date to the nearest prior business day if it falls on a weekend.
    """
    try:
        # Convert purchase date to datetime object
        buy_date = datetime.strptime(buy_date, '%Y-%m-%d')

        if ticker == 'CL=F':  # Crude Oil (WTI)
            # Start with next delivery month
            delivery_month = (buy_date.month % 12) + 1
            year = buy_date.year + (1 if delivery_month == 1 else 0)

            # Handle case where delivery_month - 1 is 0 (December of the previous year)
            expiry_year = year if delivery_month > 1 else year - 1
            expiry_month = delivery_month - 1 if delivery_month > 1 else 12

            # Calculate potential expiry (3 business days before the 25th of the prior month)
            potential_expiry = pd.Timestamp(expiry_year, expiry_month, 25) - pd.tseries.offsets.BDay(3)

            # If the expiry is before the buy date, roll to the next delivery month
            if potential_expiry < buy_date:
                delivery_month = (delivery_month % 12) + 1
                year += (1 if delivery_month == 1 else 0)
                expiry_year = year if delivery_month > 1 else year - 1
                expiry_month = delivery_month - 1 if delivery_month > 1 else 12
                potential_expiry = pd.Timestamp(expiry_year, expiry_month, 25) - pd.tseries.offsets.BDay(3)

            expiry_date = potential_expiry

        elif ticker == 'BZ=F':  # Brent Crude Oil
            # Start with next delivery month
            delivery_month = (buy_date.month % 12) + 1
            year = buy_date.year + (1 if delivery_month == 1 else 0)

            # Calculate potential expiry (2 business days before the 1st of the delivery month)
            potential_expiry = pd.Timestamp(year, delivery_month, 1) - pd.tseries.offsets.BDay(2)

            # If the expiry is before the buy date, roll to the next delivery month
            if potential_expiry < buy_date:
                delivery_month = (delivery_month % 12) + 1
                year += (1 if delivery_month == 1 else 0)
                potential_expiry = pd.Timestamp(year, delivery_month, 1) - pd.tseries.offsets.BDay(2)

            expiry_date = potential_expiry

        elif ticker == 'NG=F':  # Natural Gas
            # Start with next delivery month
            delivery_month = (buy_date.month % 12) + 1
            year = buy_date.year + (1 if delivery_month == 1 else 0)

            # Calculate potential expiry (3 business days before the 1st of the delivery month)
            potential_expiry = pd.Timestamp(year, delivery_month, 1) - pd.tseries.offsets.BDay(3)

            # If the expiry is before the buy date, roll to the next delivery month
            if potential_expiry < buy_date:
                delivery_month = (delivery_month % 12) + 1
                year += (1 if delivery_month == 1 else 0)
                potential_expiry = pd.Timestamp(year, delivery_month, 1) - pd.tseries.offsets.BDay(3)

            expiry_date = potential_expiry

        elif ticker == 'HO=F':  # Heating Oil
            # Start with next delivery month
            delivery_month = (buy_date.month % 12) + 1
            year = buy_date.year + (1 if delivery_month == 1 else 0)

            # Handle case where delivery_month - 1 is 0 (December of the previous year)
            expiry_year = year if delivery_month > 1 else year - 1
            expiry_month = delivery_month - 1 if delivery_month > 1 else 12

            # Calculate potential expiry (last business day of the prior month)
            potential_expiry = pd.Timestamp(expiry_year, expiry_month, 1) - pd.tseries.offsets.BDay(1)

            # If the expiry is before the buy date, roll to the next delivery month
            if potential_expiry < buy_date:
                delivery_month = (delivery_month % 12) + 1
                year += (1 if delivery_month == 1 else 0)
                expiry_year = year if delivery_month > 1 else year - 1
                expiry_month = delivery_month - 1 if delivery_month > 1 else 12
                potential_expiry = pd.Timestamp(expiry_year, expiry_month, 1) - pd.tseries.offsets.BDay(1)

            expiry_date = potential_expiry

        else:
            raise ValueError(f"Unsupported ticker: {ticker}. Supported tickers are: CL=F, BZ=F, NG=F, HO=F.")

        # Return expiry date
        return expiry_date.strftime('%Y-%m-%d')

    except Exception as e:
        return f"Error calculating expiry date: {e}"

# Example usage
buy_date = "2024-12-29"  # Replace with the date of purchase
ticker = "NG=F"  # Replace with 'BZ=F', 'NG=F', or 'HO=F'
expiry_date = get_energy_expiry(buy_date, ticker)
print(f"The expiry date for the {ticker} futures contract purchased on {buy_date} is {expiry_date}.")


The expiry date for the NG=F futures contract purchased on 2024-12-29 is 2025-01-29.


In [89]:
from datetime import datetime
import pandas as pd

def get_futures_expiry(buy_date, ticker):
    """
    Calculate the expiry date of the front-month futures contract for commodities based on the date of purchase.

    Args:
        buy_date (str): The purchase date in 'YYYY-MM-DD' format.
        ticker (str): The commodity ticker (e.g., 'CL=F', 'BZ=F', 'NG=F', 'HO=F', 'ZS=F', 'ZW=F', 'ZC=F', 'CC=F).

    Returns:
        str: Expiry date of the front-month contract in 'YYYY-MM-DD' format.

    Logic:
        1. Identify the commodity type based on the ticker.
        2. Define expiry rules for the identified commodity.
        3. Determine delivery month based on purchase date.
        4. Handle cases where the expiry of the next delivery month is before the buy date.
        5. Calculate expiry date using specific commodity rules.
        6. Adjust expiry date to the nearest prior business day if it falls on a weekend.
    """
    # Define valid contract months for CBOT commodities
    cbot_contract_months = {
        'ZS=F': [1, 3, 5, 7, 8, 9, 11],  # Soybean
        'ZW=F': [3, 5, 7, 9, 12],        # Wheat
        'ZC=F': [3, 5, 7, 9, 12],        # Corn
        'CC=F': [3, 5, 7, 9, 12],        # Cocoa
    }

    try:
        # Convert purchase date to datetime object
        buy_date = datetime.strptime(buy_date, '%Y-%m-%d')

        if ticker in cbot_contract_months:  # CBOT Commodities
            contract_months = cbot_contract_months[ticker]

            # Determine the front-month contract
            year = buy_date.year
            month = buy_date.month

            # Find the nearest valid contract month
            for contract_month in contract_months:
                if contract_month >= month:
                    front_month = contract_month
                    break
            else:
                # If no valid month in current year, roll to the next year's first contract month
                front_month = contract_months[0]
                year += 1

            # Calculate the expiry date for the front-month contract
            expiry_date = datetime(year, front_month, 15) - pd.tseries.offsets.BDay(1)

            # Define a rollover period (e.g., 20 business days before expiry)
            rollover_date = expiry_date - pd.tseries.offsets.BDay(20)

            # If buy_date is on or after the rollover date, move to the next contract
            if buy_date >= rollover_date:
                next_index = contract_months.index(front_month) + 1
                if next_index >= len(contract_months):
                    front_month = contract_months[0]
                    year += 1
                else:
                    front_month = contract_months[next_index]

                # Recalculate expiry date for the new contract
                expiry_date = datetime(year, front_month, 15) - pd.tseries.offsets.BDay(1)

        elif ticker in ['CL=F', 'BZ=F', 'NG=F', 'HO=F']:  # Energy Commodities
            # Start with next delivery month
            delivery_month = (buy_date.month % 12) + 1
            year = buy_date.year + (1 if delivery_month == 1 else 0)

            if ticker == 'CL=F':  # Crude Oil (WTI)
                expiry_year = year if delivery_month > 1 else year - 1
                expiry_month = delivery_month - 1 if delivery_month > 1 else 12
                potential_expiry = pd.Timestamp(expiry_year, expiry_month, 25) - pd.tseries.offsets.BDay(3)

            elif ticker == 'BZ=F':  # Crude Oil (Brent)
                potential_expiry = pd.Timestamp(year, delivery_month, 1) - pd.tseries.offsets.BDay(2)

            elif ticker == 'NG=F':  # Natural Gas
                potential_expiry = pd.Timestamp(year, delivery_month, 1) - pd.tseries.offsets.BDay(3)

            elif ticker == 'HO=F':  # Heating Oil
                expiry_year = year if delivery_month > 1 else year - 1
                expiry_month = delivery_month - 1 if delivery_month > 1 else 12
                potential_expiry = pd.Timestamp(expiry_year, expiry_month, 1) - pd.tseries.offsets.BDay(1)

            # If the expiry is before the buy date, roll to the next delivery month
            if potential_expiry < buy_date:
                delivery_month = (delivery_month % 12) + 1
                year += (1 if delivery_month == 1 else 0)
                if ticker == 'CL=F':
                    expiry_year = year if delivery_month > 1 else year - 1
                    expiry_month = delivery_month - 1 if delivery_month > 1 else 12
                    potential_expiry = pd.Timestamp(expiry_year, expiry_month, 25) - pd.tseries.offsets.BDay(3)
                elif ticker == 'BZ=F':
                    potential_expiry = pd.Timestamp(year, delivery_month, 1) - pd.tseries.offsets.BDay(2)
                elif ticker == 'NG=F':
                    potential_expiry = pd.Timestamp(year, delivery_month, 1) - pd.tseries.offsets.BDay(3)
                elif ticker == 'HO=F':
                    expiry_year = year if delivery_month > 1 else year - 1
                    expiry_month = delivery_month - 1 if delivery_month > 1 else 12
                    potential_expiry = pd.Timestamp(expiry_year, expiry_month, 1) - pd.tseries.offsets.BDay(1)

            expiry_date = potential_expiry

        else:
            raise ValueError(f"Unsupported ticker: {ticker}. Supported tickers are: CL=F, BZ=F, NG=F, HO=F, ZS=F, ZW=F, ZC=F.")

        # Return expiry date
        return expiry_date.strftime('%Y-%m-%d')

    except Exception as e:
        return f"Error calculating expiry date: {e}"

# Example usage
buy_date = "2024-12-29" 
ticker = "ZC=F"
expiry_date = get_futures_expiry(buy_date, ticker)
print(f"The expiry date for the {ticker} futures contract purchased on {buy_date} is {expiry_date}.")


The expiry date for the ZC=F futures contract purchased on 2024-12-29 is 2025-03-14.
